<a href="https://colab.research.google.com/github/ADITYAPANJIYAR/Final-Year-Project/blob/main/Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the libraries
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
classifier= Sequential() # Initialise the CNN

In [ ]:

classifier.add(Convolution2D(filters=32, 
                             kernel_size=(3,3),
                             input_shape=(64, 64, 3), 
                             kernel_initializer='he_uniform', 
                             activation='relu'))

In [ ]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
classifier.add(Convolution2D(filters=32,
                             kernel_size=(3,3), 
                             kernel_initializer='he_uniform', 
                             activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
classifier.add(Flatten())

In [ ]:
classifier.add(Dense(units=128,kernel_initializer='glorot_uniform', activation='relu')) 
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [ ]:
test_datagen= ImageDataGenerator(rescale=1./255)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/Project image dataset')

In [ ]:
training_set= train_datagen.flow_from_directory(directory= 'training_set',
                                               target_size=(64,64), # As we choose 64*64 for our convolution model
                                               batch_size=50,
                                               class_mode='binary' # for 2 class binary 
                                               )

Found 1883 images belonging to 5 classes.


In [ ]:
test_set= test_datagen.flow_from_directory(directory= 'test_set',
                                               target_size=(64,64), # As we choose 64*64 for our convolution model
                                               batch_size=50,
                                               class_mode='binary' # for 2 class binary
                                          )

Found 439 images belonging to 5 classes.


In [ ]:
from keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filename
checkpoint_path = '/content/drive/MyDrive/Project image dataset/model.h5'

# Configure the model checkpoint callback
checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# Add the checkpoint callback to the training process
classifier.fit_generator(
    training_set,
    steps_per_epoch=7,
    epochs=269,
    validation_data=test_set,
    validation_steps=1883,
    callbacks=[checkpoint]
)


In [ ]:
import os
print(os.getcwd())


/content/drive/MyDrive/Project image dataset


In [ ]:
import os

os.chdir('/content/drive/MyDrive/Project image dataset')

files = os.listdir()
print(files)


['training_set', 'test_set']


In [ ]:
from keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filename
checkpoint_path = '/content/drive/MyDrive/Project image dataset/model.h5'

# Configure the model checkpoint callback
checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# Add the checkpoint callback to the training process
classifier.fit_generator(
    training_set,
    steps_per_epoch=7,
    epochs=269,
    validation_data=test_set,
    validation_steps=1883,
    callbacks=[checkpoint]
)


In [ ]:
import os
from keras.models import load_model

# Directory to search for the model file
directory = 'path_to_directory'

# Search for the model file in the directory and its subdirectories
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.h5'):
            model_path = os.path.join(root, file)
            print("Found model at:", model_path)
            classifier = load_model(model_path)
            break  # Stop searching after finding the first model

# Continue with the rest of your code for image testing and predictions...


In [ ]:
from PIL import Image
import numpy as np
from keras.models import load_model

# Load the pre-trained model
classifier = load_model('/content/drive/MyDrive/Project image dataset/model.h5')

# Define class indices
class_indices = {
    0: 'Animal',
    1: 'Bird',
    2: 'Human',
    3: 'Nature',
    4: 'Vehicle'
}

# Array of image paths
image_paths = ["/content/3615.png",
               "/content/3724.png",
               "/content/3764.png",
               "/content/3791.png",
               "/content/3987.png",
               "/content/4087.png",
               "/content/4113.png",
               "/content/4170.png",
               "/content/4273.png",
               "/content/4276.png"
               ]

# Iterate over the image paths
for img_path in image_paths:
    # Load and preprocess the image
    test_image = Image.open(img_path)
    test_image = test_image.resize((64, 64))  # Resize the image to the desired size
    test_image = np.array(test_image)  # Convert the image to a numpy array
    test_image = test_image / 255.0  # Normalize the image
    test_image = np.expand_dims(test_image, axis=0)  # Add an extra dimension for batch

    # Make predictions
    result = classifier.predict(test_image)

    # Determine the predicted label
    predicted_class_index = np.argmax(result)
    prediction = class_indices[predicted_class_index]

    # Print the predicted label
    print(f" Prediction: {prediction}")